In [1]:
import torch
import argparse
from model import GPT
from utils import generate
from utils import getLoaderDataset
from utils import Args
from train import Training

args = Args()
print(args)
train = Training(args)
model, metrics = train.train_model()
print(train.tokenized_data.decode(
    generate(model, train.tokenized_data.encode("Oh God Oh God !"), 50)
))

Args(batch_size=10, n_tokens=64, n_layers=4, n_heads=4, d_model=128, use_lr_decay=True, lr=0.001, dataset_path='./datasets/shakespear_corpus.txt', max_iter=100, out_dir=None, n_warm_iters=100, lr_decay_iter=5000, min_lr=0.0001, n_validation_batch=200, betas=(0.9, 0.99), n_epochs=10, val_int=100)
Epoch: 0, Batch 99, Training Loss: 2.5976, Validation Loss: 2.5998
Oh God Oh God !
O:
fith ove harnghishe th ham, beind,he.
Sorsfert


/home/gordon/Documents/edu/gordon_ms/Sem3/DeepLearning/Shakespeak/utils.py:270: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_c = torch.tensor(char_id).reshape(shape=(1, 1))


In [ ]:
N = 64
B = 12
loader, dataset = getLoaderDataset(N, B, "./datasets/shakespear_corpus.txt")

In [ ]:
N = 64
B = 12
loader, token_dataset = getLoaderDataset(N, B, "./datasets/shakespear_corpus.txt")
print(len(token_dataset))

1003853 64
1003853 64
1003853 64
1003789


In [ ]:
L = 12
d = 768
d_ff = 4 * d
h = 8
V = dataset.get_vocab_size()
print(V)
model = GPT(B, L, d, d_ff, N, h, V)

In [18]:
new_tokens = generate(model, dataset.encode("Hello World"), 100)
print(dataset.decode(new_tokens))

In [ ]:
N = 64
B = 12
L = 6
d = 128
d_ff = 4 * d
h = 6
V = dataset.get_vocab_size()

In [ ]:
B = 12
N = 64  # context of up to 256 previous characters
L = 4
h = 4
d = 128
learning_rate = 1e-3
use_lr_decay = False
dataset = './datasets/shakespear_corpus.txt'
out_dir = './runs/'

max_iterations = 1000

class Args(argparse.Namespace):
    batch_size:int = B
    n_tokens:int = N
    n_layers:int = L
    n_heads:int = h
    d_model:int = d
    use_lr_decay:bool = use_lr_decay
    learning_rate: float = learning_rate
    dataset: str = dataset
    max_iterations: int = max_iterations
    out: str = out_dir

In [ ]:
model, losses, perplexities = train_model(Args())

In [ ]:
import torch

B = 12
N = 64  # context of up to 256 previous characters
L = 4
h = 4
d = 128
learning_rate = 1e-3
betas = (0.9, 0.99)
eps = 10e-9
n_warmup_iterations = 100
learning_rate_decay_iterations = 5000
min_learning_rate = 1e-4
use_lr_decay = True
loader, dataset = getLoaderDataset(N, B, "./datasets/shakespear_corpus.txt")

model = GPT(B, L, d, 3*d, N, h, V)

cpkt = torch.load("./runs/model_10000.pt", map_location=torch.device('cpu'))
model.load_state_dict(cpkt)
print(model)

In [ ]:
from utils import generate

new_tokens = generate(model, dataset.encode("Hello World"), 300)
print(dataset.decode(new_tokens))